Vi starter med at definere en funktion for vores forsøg. En måling defineres som at slå med en terning N gange og lægge øjnene sammen. Dette gentages så 10 000 gange.

In [ ]:
import numpy as np

np.random.seed(1)

# Kast en terning N gange og lægger øjnene sammen. Gentager dette 10 000 gange
def diceroll(N):
    # Terningens egenskaber
    muligheder = [1,2,3,4,5,6]
    # Jeg har valgt ikke at vægte terningen, da det var svært at fitte normalfordelingen til en vægtet terning
    sandsynligheder = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
    # Laver N terningekast 10 000 gange
    kast_arr = np.random.choice(muligheder, (10000, N), p=sandsynligheder)
    # Lægger øjnene sammen
    return np.sum(kast_arr, axis = 1)
    



Tæthedsfunktionen for normalfordelingen.

In [ ]:
# Normalfordelingen
def normal_distribution(x, sigma, mu, k):
    return (1/(sigma * np.sqrt(2*np.pi)))*np.exp((-(x-mu)**2)/2*(sigma**2)) * k


Hjælpefunktion til at afrunde til korrekt antal betydende cifre baseret på usikkerheden.\
\
Hentet fra [dette spørgsmål](https://stackoverflow.com/questions/53976847/report-uncertainty-given-a-mean-and-the-standard-error-show-only-significant-f) på stackoverflow.\
\
Der findes pakker der kan gøre det, men jeg var ikke sikker på om det ville virke på en anden computer så jeg bruger denne metode i stedet for.

In [ ]:
from decimal import Decimal
from IPython.display import Markdown as md, display_markdown

def round_uncert(value, uncertainty):
    # Fjerner nuller bag det sidste ciffer i usikkerheden
    u = Decimal(uncertainty).normalize()
    # Finder positionen af det første ciffer i usikkerheden
    exponent = u.adjusted()
    # Boolean der gemmer om det første ciffer af usikkerheden er 1
    precision = (u.as_tuple().digits[0] == 1)
    # Usikkerheden skaleres så dens første ciffer er det første/eneste tal før kommaet
    # Hvis det første ciffer er 1 afrundes til 1 decimal, ellers afrundes der til 0 decimaler.
    u = u.scaleb(-exponent).quantize(Decimal(10)**-precision)

    # Tallet skalere til samme tierpotens som usikkerheden og afrundes
    # Returnerer tallet, usikkerheden og tierpotens
    return round(Decimal(value).scaleb(-exponent).quantize(u)), u, exponent


Denne blok laver histogram/normalfordelingsfit for et givent N og plotter det. Desuden udregnes middelværdien.\
\
Bemærk at vi fitter til venstre kant af vores bins, da det får fittets middelværdi til at stemme overens med middelværdien fra np.mean(). Hvis vi fittede til midten af binsne ville toppunktet af tæthedsfunktionen være forskudt 0.5 til højre for middelværdien, hvilket ville se grimt ud når vi plotter det.

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def dice_data(N):
    data = diceroll(N)

    # Udregn antal bins
    bin_num = np.arange(np.min(data),np.max(data)+2)

    # Lav histogrammet
    counts, bin_edges, patches = plt.hist(data, bins=bin_num, color="skyblue", alpha=0.8, label="Histogram", align="left")

    # Fjern den sidste kant for at have N kanter at fitte til 
    # (den sidste kant er højre kant på den sidste bin)
    bin_corners = np.delete(bin_edges, -1)

    # Fit tæthedsfunktionen fra normalfordelingen til histogrammet
    # Startgættene fandt jeg ved at prøve mig frem
    if N == 1:
        starting_guess = [0.1, np.mean(data), 0.1]
    else:
        starting_guess = [1, np.mean(data), 1]

    par, cov = curve_fit(normal_distribution, bin_corners, counts, p0=starting_guess, maxfev=100000)
    
    # Middelværdi og standardafvigelse findes ud fra parametrene fra fittet
    # Find middelværdi, standardafvigelse og usikkerhed på middelværdien
    mean = np.mean(data)
    std = np.std(data)
    mean_uncertainty = std / np.sqrt(len(data))

    rounded_mean = round_uncert(mean, mean_uncertainty)

    # Plot middelværdien med usikkerhed
    plt.axvline(mean, color="b", label="Middelværdi")
    plt.axvline(mean - mean_uncertainty, color="cyan", label="Usikkerhed på middelværdien")
    plt.axvline(mean + mean_uncertainty, color="cyan")
    
    # Plot tæthedsfunktionen
    X = np.linspace(0, max(bin_edges + 2), 100*len(data))
    plt.plot(X, normal_distribution(X, *par), color="r", linestyle='dashed', label="Fit til normalfordelingen")

    plt.xlim(min(bin_edges - 1), max(bin_edges + 1))
    plt.title("Histogram over summer med N = {}".format(N))
    plt.grid()
    plt.legend()
    plt.show()

    # Vis middelværdi med usikkerhed
    display_markdown(md(r"Middelværdi af summen for {} kast: $({} \pm {}) \cdot 10^{}$".format(N, rounded_mean[0], str(rounded_mean[1]), rounded_mean[2])))




For at få konsekvente resultater.

In [ ]:
# ( ͡° ͜ʖ ͡°)
np.random.seed(69)

Nu mangler vi kun at plotte plotsne og vise middelværdierne.

In [ ]:
dice_data(1)
dice_data(5)
dice_data(10)
dice_data(50)